# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
SMALL_KGE_DIMENSION = 4
LARGE_KGE_DIMENSION = MODEL_HIDDEN_SIZE
KGE_DIMENSIONS = [SMALL_KGE_DIMENSION, LARGE_KGE_DIMENSION] # Output Dimension of the GNN Encoder.
EPOCHS = 1
BATCH_SIZE = 64

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,labels,prompt,split
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",1,"0[SEP]Toy Story (1995)[SEP]['Adventure', 'Anim...",val
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",1,"0[SEP]Grumpier Old Men (1995)[SEP]['Comedy', '...",train
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",1,"0[SEP]Heat (1995)[SEP]['Action', 'Crime', 'Thr...",train
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",1,0[SEP]Seven (a.k.a. Se7en) (1995)[SEP]['Myster...,test
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']",1,"0[SEP]Usual Suspects, The (1995)[SEP]['Crime',...",train


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainer =    GNNTrainer(movie_lens_loader.data, kge_dimension = SMALL_KGE_DIMENSION)
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:

print(gnn_trainer.kge_dimension)
gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
print(gnn_trainer_large.kge_dimension)
gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4


100%|██████████| 190/190 [00:03<00:00, 48.23it/s]


Epoch: 001, Loss: 0.2127


100%|██████████| 79/79 [00:00<00:00, 129.72it/s]



Validation AUC: 0.9195
128


100%|██████████| 190/190 [00:06<00:00, 28.10it/s]


Epoch: 001, Loss: 0.1084


100%|██████████| 79/79 [00:00<00:00, 94.83it/s]


Validation AUC: 0.9175


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
gnn_trainer.get_embeddings(movie_lens_loader)
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,labels,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",1,"0[SEP]Toy Story (1995)[SEP]['Adventure', 'Anim...",val,"[-1.4025567770004272, 0.6816163659095764, -3.0...","[0.7183933258056641, -0.7495675086975098, -0.5...","[-0.13522014021873474, 0.16583335399627686, 0....","[-0.15604138374328613, 0.22537894546985626, 0...."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",1,"0[SEP]Grumpier Old Men (1995)[SEP]['Comedy', '...",train,"[-1.0757461786270142, 0.6430796384811401, -2.7...","[0.8827657103538513, -0.8039241433143616, -0.1...","[-0.13852912187576294, 0.22851958870887756, 0....","[-0.5181198120117188, 0.18665221333503723, 0.4..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",1,"0[SEP]Heat (1995)[SEP]['Action', 'Crime', 'Thr...",train,"[-1.1887747049331665, 0.6766993999481201, -2.8...","[0.3578329086303711, -0.37480437755584717, -0....","[-0.1273198425769806, 0.21016445755958557, 0.3...","[0.25531184673309326, 0.1213562935590744, 0.42..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",1,0[SEP]Seven (a.k.a. Se7en) (1995)[SEP]['Myster...,test,"[-1.2844634056091309, 0.6699168682098389, -2.9...","[0.49199721217155457, -0.14119231700897217, -0...","[-0.13550306856632233, 0.21558162569999695, 0....","[0.18766692280769348, 0.1714116334915161, 0.47..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']",1,"0[SEP]Usual Suspects, The (1995)[SEP]['Crime',...",train,"[-1.028777003288269, 0.6010562181472778, -2.48...","[0.6986081004142761, -0.4587530195713043, -0.2...","[-0.2011059671640396, 0.22581297159194946, 0.3...","[0.05735247954726219, 0.24271981418132782, 0.4..."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME)

Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension)

We also train the model. This can be skipped if already done ones.

In [13]:
#prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS)

In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)

Some weights of AddingEmbeddingBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

In [16]:
#movie_lens_loader.add_false_edges(false_ratio = 2.0)

In [17]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME, false_ratio=-1)
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME, false_ratio = -1)
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME, false_ratio=-1)


Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AddingEmbeddingBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
dataset_vanilla_fixed = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function, suffix = "_fixed")
#dataset_prompt_fixed = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension, suffix = "_fixed")
#dataset_adding_embedding_fixed = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, suffix = "_fixed", kge_dimension = MODEL_HIDDEN_SIZE)

In [19]:
dataset_vanilla_fixed["train"][0]["ranges_over_batch"]

[[1, 2], [2, 3], [3, 11], [11, 12], [12, 21], [21, 22]]

In [20]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(dataset_vanilla_fixed, epochs=1)
#prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt_fixed, epochs=1)
#df = adding_embedding_bert_only_classifier.forward_dataset_and_save_outputs(dataset_adding_embedding_fixed, epochs=1)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Vanilla train Forward Epoch 1 from 1
Vanilla test Forward Epoch 1 from 1
Vanilla val Forward Epoch 1 from 1


,level_0,index,user_id,title,genres,labels,split,hidden_states,attentions
0,0,0,0,grumpier old men ( 1995 ),"['comedy ', 'romance']",1,train,"[[tensor(0.0433), tensor(-0.4525), tensor(-0.9...","[[[tensor(0.6167), tensor(0.2433)], [tensor(0...."
1,1,1,0,heat ( 1995 ),"['action ', 'crime ', 'thriller']",1,train,"[[tensor(-0.0737), tensor(-0.4414), tensor(-1....","[[[tensor(0.6005), tensor(0.2771)], [tensor(0...."
2,2,2,0,"usual suspects, the ( 1995 )","['crime ', 'mystery ', 'thriller']",1,train,"[[tensor(0.1260), tensor(-0.4831), tensor(-1.2...","[[[tensor(0.5491), tensor(0.2855)], [tensor(0...."
3,3,3,0,bottle rocket ( 1996 ),"['adventure ', 'comedy ', 'crime ', 'romance']",1,train,"[[tensor(-0.1080), tensor(-0.4961), tensor(-1....","[[[tensor(0.5437), tensor(0.2620)], [tensor(0...."
4,4,4,0,canadian bacon ( 1995 ),"['comedy ', 'war']",1,train,"[[tensor(0.3179), tensor(-0.2600), tensor(-0.9...","[[[tensor(0.6132), tensor(0.2528)], [tensor(0...."
...,...,...,...,...,...,...,...,...,...
272254,51421,29,314,bee movie ( 2007 ),"['animation ', 'comedy']",0,val,"[[tensor(-1.0330), tensor(0.3144), tensor(-0.5...","[[[tensor(0.9281), tensor(0.0470)], [tensor(0...."
272255,51422,30,404,goodbye lover ( 1999 ),"['comedy ', 'crime ', 'thriller']",0,val,"[[tensor(-1.0966), tensor(0.9011), tensor(-0.3...","[[[tensor(0.7357), tensor(0.3178)], [tensor(0...."
272256,51423,31,221,blues brothers 2000 ( 1998 ),"['action ', 'comedy ', 'musical']",0,val,"[[tensor(-1.0801), tensor(0.1345), tensor(-0.3...","[[[tensor(0.8995), tensor(0.1729)], [tensor(0...."
272257,51424,32,279,exam ( 2009 ),"['mystery ', 'thriller']",0,val,"[[tensor(-0.8209), tensor(0.9609), tensor(-0.5...","[[[tensor(0.7075), tensor(0.1596)], [tensor(0...."
